In [1]:
import folium
from function import folium_functions

global_path = '../data/London_in_depth/'
output_map_name = global_path + "london_tubes.html"

# Load data

In [2]:
# Files containing the position of the London underground stations and lines
filename_stations = global_path + 'london_stations.csv'
filename_lines = global_path + 'london_tube_lines.csv'

# Initialize the map

In [3]:
city_of_london_coordinates = [51.51279, -0.09184]
m = folium.Map(location=city_of_london_coordinates, zoom_start=12)
m

# Add the stations

In [4]:
dic_stations = folium_functions.pipeline_stations(filename_stations, max_zone=2, current_map=m)
m

# Add the lines

In [5]:
folium_functions.pipeline_lines(filename_lines, filename_stations, m)
m